<a href="https://colab.research.google.com/github/klattery/Stan/blob/master/Stan_Setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1) Install cmdstanr & Get system info**

In [ ]:
library(devtools)
devtools::install_github("stan-dev/cmdstanr", dependencies=c("Depends", "Imports"))
install.packages("benchmarkme")
library(cmdstanr); library(posterior); library(benchmarkme)
Sys.info();get_cpu();get_ram()

**2) Install Stan**

In [ ]:
if (!file.exists("cmdstan-2.23.0.tgz")) {
  system("wget https://github.com/stan-dev/cmdstan/releases/download/v2.23.0/colab-cmdstan-2.23.0.tar.gz", intern=T)
  system("tar zxf colab-cmdstan-2.23.0.tar.gz", intern=T)
}
set_cmdstan_path("cmdstan-2.23.0")

**3) Choose Model and Load Data**

In [ ]:
HB_model <- cmdstan_model("MNL_BartBlockCon_v1_7.stan", quiet = TRUE)
load("data_conjoint.RData") # Uploaded data to Colab

**4) Specify constraints and HB specfics**

In [ ]:
# Specify constraints (sign only)
# For each parameter: 0 = no constraint, +1 = positive, -1 = negative
P <- data_conjoint$P
con_sign <- rep(0,P)
con_sign[18:21] <- -1 # Negative utilities for price slopes

# Modeling parameters. This overwrites/adds to the data file we pass to Stan.
data_model <- list(
  con_sign = con_sign,
  prior_cov = data_conjoint$prior_cov * 1, # Change cov scale here  
  df = 5, # Degrees of freedom
  prior_alpha = rep(0, P),
  a_sig = 10,
  cov_block = matrix(1, P, P),
  splitsize = round(.5 + data_conjoint$T/(4 * threads[[2]])),
  agg_model = NULL,
)

**Run model**

In [ ]:
outname <- paste0("Stan_BaseMNL-", # edit as desired
                  format(Sys.time(), '%Y%m%d-%H%M%S')) 
HB_model$sample(modifyList(data_conjoint, data_model),
                iter_warmup = 200,
                iter_sampling = 200,
                output_dir ="/content",
                output_basename = outname,
                chains = 2,
                save_warmup = TRUE,
                refresh = 10,
                seed = 271,
                init = .1,
                show_messages = FALSE,
                validate_csv = FALSE
)

**`Check Convergence`**

In [ ]:
library("posterior")
csv_name <- c(paste0(outname,"-1.csv"),
              paste0(outname,"-2.csv")
) # Output files in path: assumes you have two chains
draws_upper <- read_cmdstan_csv(file.path("/content", csv_name), variables = c("alpha"))
fit_stats <- summarize_draws(draws_upper$post_warmup_draws)
fit_stats$variable <- colnames(data_conjoint$ind)
fit_stats <- fit_stats[,-4:-5]
fit_stats[,-1] <- round(fit_stats[,-1],2)
fit_stats

Plot Mean Utilities by Iteration and Compute Point Estimates

In [ ]:
draws_beta <- read_cmdstan_csv(file.path("/content", csv_name), variables = "beta_ind", sampler_diagnostics = "")
ndraws <- dim(draws_beta$post_warmup_draws)[1]
draws_beta_mu1 <- t(sapply(1:ndraws, function(i){
  beta_mu <- colMeans(matrix(draws_beta$post_warmup_draws[i,1,],data_conjoint$I,data_conjoint$P, byrow = TRUE))  
}))
draws_beta_mu2 <- t(sapply(1:ndraws, function(i){
  beta_mu <- colMeans(matrix(draws_beta$post_warmup_draws[i,2,],data_conjoint$I,data_conjoint$P, byrow = TRUE))  
}))
for (i in 1:ncol(draws_beta_mu1)){
  x <- cbind(draws_beta_mu1[,i],draws_beta_mu2[,i])
  plot(x[,1], type = "l", col = "red", main = paste("Mean Beta ", colnames(data_conjoint$ind)[i],
                                                    "| rhat = ", round(rhat(x),2),
                                                    "| ESS = ", round(ess_basic(x),1)
  ))
  lines(x[,2], type = "l", col = "blue")
}
utilities <- matrix(colMeans(draws_beta$post_warmup_draws, dims = 2),
                    data_conjoint$I, data_conjoint$P, byrow = TRUE)
write.table(utilities, file = file.path("/content", "beta_final_r.csv"), sep = ",", na = ".", row.names = FALSE)
